# 2D Fourier Transform

In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
images = [ cv2.imread("./images/data/00{}.jpg".format(i), cv2.IMREAD_GRAYSCALE) for i in range(0, 7)] 

In [2]:
def fft2(img: np.array): 
    M, N = img.shape 
    fourier = np.matmul(exp_mu(img), np.matmul(img, exp_nu(img)) )
    return fourier

def inverse_fft2(img: np.array):
    M, N = img.shape 
    inv_fourier = np.matmul(exp_mu(img, inverse = True), np.matmul(img , exp_nu(img, inverse = True))) / (M * N)
    return inv_fourier

def highpass(img, radius = 45): 
    h, w = img.shape[0], img.shape[1]
    filtered = [[0 for _ in range(w)] for _ in range(h)]
    for i in range(h): 
        for j in range(w):
            distance = ((i - h // 2) ** 2 + (j - w // 2) ** 2)
            if distance < radius ** 2:
                continue 
            filtered[i][j] = img[i][j]
    return np.array(filtered)
    
    
def exp_mu(img: np.array, inverse = False):
    M = img.shape[0]
    result = [[0 for _ in range(M)] for _ in range(M)]
    for u in range(M):
        for m in range(M):
            if inverse:
                result[u][m] = np.exp(2j* np.pi * m * u / M )
            else: 
                result[u][m] = np.exp(-2j* np.pi * m * u / M )
    return np.array(result)

def exp_nu(img: np.array, inverse = True):
    N = img.shape[1]
    result = [[0 for _ in range(N)] for _ in range(N)]
    for v in range(N):
        for n in range(N):
            if inverse:
                result[v][n] = np.exp(2j* np.pi * n * v / N )
            else:
                result[v][n] = np.exp(-2j* np.pi * n * v / N )  
    return np.array(result)

def shift(img: np.array):    
    return np.fft.fftshift(img)

def fft_shift(img: np.array):
    return np.abs(shift(fft2())).round().astype(np.uint8)

def show(img: np.array):
    plt.imshow(fft_shift(img), cmap='gray')
    

## Azimuthal Averaging

In [3]:
def azimuthal_averaging(img: np.ndarray):
    h, w = img.shape 
    center = (h // 2, w // 2) 
    max_radius = np.sqrt(center[0] ** 2 + center[1] ** 2) 
    if max_radius != int(max_radius):
        max_radius += 1
    max_radius = int(max_radius)
    cum_sum_freq = np.array([0 for _ in range(max_radius)])
    pixels = np.array([0 for _ in range(max_radius)])
    for i in range(h):
        for j in range(w):
            radius = int(np.sqrt((i - center[0]) ** 2 + (j - center[1]) ** 2))
            cum_sum_freq[radius] += img[i][j]
            pixels[radius] += 1
    cum_sum_freq = cum_sum_freq / pixels 
    cum_sum_freq = cum_sum_freq[::-1]
    return cum_sum_freq
    

In [132]:
def pipeline(img, name, center = False,  suffix = ""): 
    h, w = img.shape
    if center:
        suffix = "_centered"
        img = img[np.ix_(range(h // 4 , 3 * h // 4), range(w // 4, 3 * w // 4))]
    print(img.shape)
        
    fft_img = fft2(img) 
    fft_shifted = np.fft.fftshift(fft_img)
    highpass_img = highpass(fft_shifted, radius = 45)
    plt.imshow(np.log(np.abs(highpass_img)), cmap='gray')
    plt.savefig("./images/frequency/00{}{}.jpg".format(name, suffix))
    plt.clf()
    
    inv_fft_img = inverse_fft2(highpass_img)
    plt.imshow(np.abs(inv_fft_img), cmap='gray')
    plt.savefig("./images/highpassed/00{}{}.jpg".format(name, suffix))
    plt.clf()
    
    return np.abs(inv_fft_img)
    

In [160]:
result =[]
for i in range(7):
    img = pipeline(images[i], i, center = False)
    result.append(img)

res = [] 

for img in result: 
    h, w = img.shape 
    val = 0
    for i in range(h):
        for j in range(w):
            val += img[i][j] * np.exp(-np.sqrt((i - h // 2) ** 2 +  (j - w // 2) ** 2))
            
    res.append(val)

(332, 332)


/var/folders/x1/tylp6nfj5xnb74kpdk36gzzw0000gn/T/ipykernel_41317/4147936517.py:11: RuntimeWarning: divide by zero encountered in log
  plt.imshow(np.log(np.abs(highpass_img)), cmap='gray')


(374, 260)
(302, 302)
(306, 298)
(318, 318)
(332, 332)
(316, 308)


<Figure size 640x480 with 0 Axes>

In [164]:
centered_result = []
for i in range(7):
    img = pipeline(images[i], i, center = True)
    centered_result.append(img)

centered_res = [] 

for img in centered_result: 
    h, w = img.shape 
    val = 0
    for i in range(h):
        for j in range(w):
            val += img[i][j] * np.exp(-np.sqrt((i - h // 2) ** 2 +  (j - w // 2) ** 2))
            
    centered_res.append(val)

(166, 166)


/var/folders/x1/tylp6nfj5xnb74kpdk36gzzw0000gn/T/ipykernel_41317/4147936517.py:11: RuntimeWarning: divide by zero encountered in log
  plt.imshow(np.log(np.abs(highpass_img)), cmap='gray')


(187, 130)
(151, 151)
(153, 149)
(159, 159)
(166, 166)
(158, 154)


<Figure size 640x480 with 0 Axes>

In [167]:
res

[8.006833820733114,
 2.046315945387563,
 9.691831836819809,
 19.453395251597062,
 21.200994740227998,
 4.143174289671995,
 25.76897682311935]

In [168]:
centered_res

[5.502374353254851,
 1.589088533510345,
 2.1940765105225637,
 6.345025710244663,
 6.0616962690927325,
 2.2441757837065146,
 15.373336955214201]